In [18]:
from pydub import AudioSegment
import os
if not os.path.exists("aegithalos_caudatus"):
    os.makedirs("aegithalos_caudatus")

count=1
for i in range(1,30,1):
    t1 = i * 1000 #Works in milliseconds
    t2 = (i+1) * 1000
    newAudio = AudioSegment.from_wav("data/train/train_aegithalos_caudatus.wav")
    newAudio = newAudio[t1:t2]
    newAudio.export('aegithalos_caudatus/'+str(count)+'.wav', format="wav") #Exports to a wav file in the current path.
    print(count)
    count+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [17]:
if not os.path.exists("alauda_arvensis"):
    os.makedirs("alauda_arvensis")

count=1
for i in range(1,30,1):
    t1 = i * 1000 #Works in milliseconds
    t2 = (i+1) * 1000
    newAudio = AudioSegment.from_wav("data/train/train_alauda_arvensis.wav")
    newAudio = newAudio[t1:t2]
    newAudio.export('alauda_arvensis/'+str(count)+'.wav', format="wav") #Exports to a wav file in the current path.
    print(count)
    count+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [19]:
from scipy.io.wavfile import read
import matplotlib.pyplot as plt
from os import walk
import os
if not os.path.exists("aegithalos_caudatus_plots"):
    os.makedirs("aegithalos_caudatus_plots")
car_wavs = []
for (_,_,filenames) in walk('aegithalos_caudatus'):
    car_wavs.extend(filenames)
    break
for car_wav in car_wavs:
    # read audio samples
    input_data = read("aegithalos_caudatus/" + car_wav)
    audio = input_data[1]
    # plot the first 1024 samples
    plt.plot(audio)
    # label the axes
    plt.ylabel("Amplitude")
    plt.xlabel("Time")
    # set the title
    # plt.title("Sample Wav")
    # display the plot
    plt.savefig("aegithalos_caudatus_plots/" + car_wav.split('.')[0] + '.png')
    # plt.show()
    plt.close('all')

In [20]:
if not os.path.exists("alauda_arvensis_plots"):
    os.makedirs("alauda_arvensis_plots")
car_wavs = []
for (_,_,filenames) in walk('alauda_arvensis'):
    car_wavs.extend(filenames)
    break
for car_wav in car_wavs:
    # read audio samples
    input_data = read("alauda_arvensis/" + car_wav)
    audio = input_data[1]
    # plot the first 1024 samples
    plt.plot(audio)
    # label the axes
    plt.ylabel("Amplitude")
    plt.xlabel("Time")
    # set the title
    # plt.title("Sample Wav")
    # display the plot
    plt.savefig("alauda_arvensis_plots/" + car_wav.split('.')[0] + '.png')
    # plt.show()
    plt.close('all')

In [22]:
import os
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)

574717952/574710816 [==============================] - 87s 0us/step


In [46]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [23]:
def get_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    flatten = model.predict(x)
    return list(flatten[0])

X = []
y = []

In [24]:
aegithalos_caudatus_plots = []
for (_,_,filenames) in os.walk('aegithalos_caudatus_plots'):
    aegithalos_caudatus_plots.extend(filenames)
    break

for cplot in aegithalos_caudatus_plots:
    X.append(get_features('aegithalos_caudatus_plots/' + cplot))
    y.append(0)

alauda_arvensis_plots = []
for (_,_,filenames) in os.walk('alauda_arvensis_plots'):
    alauda_arvensis_plots.extend(filenames)
    break

for cplot in alauda_arvensis_plots:
    X.append(get_features('alauda_arvensis_plots/' + cplot))
    y.append(1)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train, y_train)

predicted = clf.predict(X_test)

# get the accuracy
print (accuracy_score(y_test, predicted))

0.8888888888888888


In [34]:
print(len(X_train[2]))

25088


In [38]:
import scipy.misc
#scipy.misc.imsave('features00000.jpg', X_train[2])
scipy.misc.toimage(np.array(X_train[2]), cmin=0.0, cmax=...).save('outfile.jpg')

AttributeError: module 'scipy.misc' has no attribute 'toimage'

In [42]:
from PIL import Image
img = Image.fromarray(np.array(X_train[2])
#im.save('your_file.jpeg')
img.show()

SyntaxError: invalid syntax (<ipython-input-42-1f9b9af6f862>, line 4)

In [45]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.imshow((np.array(X_train[2]))
plt.show()

SyntaxError: invalid syntax (<ipython-input-45-840c2359583e>, line 4)